### Import library

In [1]:
""" Example 2 is inbalanced data set; ~2200 in PD and ~1100 in SNP
    Goal is to predict if mutation is SNP or PD
    CV branch
    
    Total samples: 3368
    2254 PD samples
    1111 SNP samples
"""

' Example 2 is inbalanced data set; ~2200 in PD and ~1100 in SNP\n    Goal is to predict if mutation is SNP or PD\n    CV branch\n    \n    Total samples: 3368\n    2254 PD samples\n    1111 SNP samples\n'

In [2]:
""" Imports the required libraries and packages
"""

import pandas as pd  #Import for data manipulation in dataframes
import numpy as np  #Array manipulation and calculates mean

import random as rd
import time

from sklearn.metrics import(
    matthews_corrcoef,  # MCC for evaluation
    balanced_accuracy_score, #hyperparameter evaluation
    f1_score,  #hyperparameter evaluation
    confusion_matrix,  # confusion matrix for classification evalutation
    classification_report #Return the F1, precision, and recall of a prediction
    )
from sklearn.model_selection import(
    train_test_split,  # Splits data frame into the training set and testing set
    GridSearchCV,  # Cross validation to improve hyperparameters
    RandomizedSearchCV,
    StratifiedKFold, # K-fold CV
    GroupKFold
        )
from sklearn.ensemble import RandomForestClassifier #SK learn API for classificastion random forests
from sklearn.tree import DecisionTreeClassifier #Single tree decisions 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier #allows for confidence scores to be predicted for each
np.set_printoptions(threshold=np.inf, precision=3) #full array printing

### Clean dataset in pandas

In [10]:
def Clean_data(file):
    """ Input:      file        The dataset to read

        Returns:    Output      Cleaned dataframe with numeric values for class

        Create, clean and convert dataset E2.csv to PD dataframe. Sorts alphabetically to order proteins, removes blank spaces
        and applies "One Hot Encoding" to convert classes (PD/SNP) to 1/0
    """
    df = pd.read_csv('E2.csv')

    #Remove unrequired column, replace blank spaces, reset index to run from 0
    df.dropna(inplace = True) #drop rows with missing values
    df.replace(' ', '_', regex=True, inplace=True)
    df.reset_index(drop=True, inplace = True)

#     Input = df.drop('dataset', axis =1)
    Output_encoded = pd.get_dummies(df, columns=['dataset']) #Encode the PD and SNP columns
    Output = Output_encoded.drop(['dataset_snp'],axis = 1)
#     Output = Output_encoded['dataset_pd'].copy().astype('int32') #PD = 1, SNP = 0

    return Output #Datset in alphabetical order, NaNs removed and encoded PD label

In [11]:
Output = Clean_data(file = 'E2.csv')
Output

,pdbcode:chain:resnum:mutation,Binding,SProtFT0,SProtFT1,SProtFT2,SProtFT3,SProtFT4,SProtFT5,SProtFT6,SProtFT7,...,NLargest6,NLargest7,NLargest8,NLargest9,NLargest10,Clash,Glycine,Proline,CisPro,dataset_pd
0,3odq:B:90:K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,35.332,33.811,30.525,28.448,23.301,-8.83,-100.0000,-100.0000,0.0,1
1,3thc:A:214:Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,152.367,142.055,119.325,110.986,110.768,-11.19,-100.0000,-100.0000,0.0,1
2,2ary:A:103:A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,74.267,69.595,67.078,62.108,61.438,-2.03,-100.0000,-100.0000,0.0,0
3,2r3v:A:301:T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,162.474,130.995,114.245,113.635,87.611,1.32,-100.0000,-100.0000,0.0,1
4,1bhg:A:408:S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,188.020,176.892,175.881,160.863,160.839,-2.93,-100.0000,-100.0000,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3360,3cot:A:261:C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,103.081,85.249,83.669,77.663,73.123,-4.57,-100.0000,-100.0000,0.0,1
3361,2l0w:A:47:V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,53.941,42.229,27.331,25.610,25.432,-5.44,2.4896,-100.0000,0.0,1
3362,2l9n:A:126:T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,73.001,59.724,56.923,50.393,47.249,-4.98,-100.0000,-100.0000,0.0,1
3363,2xwt:C:252:P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,62.633,51.638,51.579,50.647,49.074,41.29,-100.0000,3.2714,0.0,1


### Split dataset into training and validation sets

In [12]:
def train(Output):
    """      
        Input:      Output          Cleaned dataframe

        Returns:    Training_Set     Training data
                    Testing_Set      Testing data

        80% training and 20% testing split. Outputs the data to files. Splits are randomly shuffled
        """
    Training_Set, Testing_Set = train_test_split(Output,train_size = 0.8)
    
    with open('Training dataset.txt', 'w') as file: #Writes training features to text file
        file.write(Training_Set.to_string())
    with open('Testing dataset.txt', 'w') as file: #Writes training class labels to text file
        file.write(Testing_Set.to_string())


    return Training_Set, Testing_Set

In [31]:
Training_Set, Testing_Set = train(Output)

### Initial evaluation

In [174]:
# def test(Initial_RFC, Input_test, Classes_test):
#     """ Input:  Input_test      Features test data
#                 Classes_test    Class label test data

#         Evaluates the training data before balancing. Random forest classifier makes prediction using the test features. True values 
#         are the class labels testing data
#     """

#     Output_pred = Initial_RFC.predict(Input_test) #Always perdict on the unseen test data, as train has been used by the estimastor
#     print(f"              **Initial Evaluation**\n")
#     print(f"Confusion Matrix:\n {confusion_matrix(Classes_test, Output_pred)}")
#     print(f"{classification_report(Classes_test, Output_pred)}\nMCC                {matthews_corrcoef(Classes_test, Output_pred)}")


### Group K-fold CV

In [32]:
Group_df = Training_Set
Group_df.sort_values(by=['pdbcode:chain:resnum:mutation'], inplace = True) #Alphabetically order PDB identifier column
Group_df.dropna(inplace = True) #Remove rows with missing values

PDB_codes = [] #Empty list that will contain only the PDB code
for i in range(len(Group_df)): 
    PDB_codes.append(Group_df.iloc[i][0].partition(':')[0]) #Obtain just the 'pdbcode' from 'pdbcode:chain:resnum:mutation'

Group_df.drop(['pdbcode:chain:resnum:mutation'], axis=1, inplace=True) #Remove 'pdbcode:chain:resnum:mutation' column
Group_df.insert(0, 'PDB code', PDB_codes) #Insert PDB codes as first column


#Groups proteins by PDB code#

PDB_codes_count = 0
PDB_codes_list = [] #Empty list that will contain the protein groups as items
for i in range(len(Group_df)-1):
    First = Group_df.iloc[i][0] #PDB code of row i
    Second = Group_df.iloc[i + 1][0] #PDB code of row i+1 

    if First == Second:
        PDB_codes_count += 1 #increase count
    else:
        PDB_codes_list.append(np.array(Group_df.iloc[(i - PDB_codes_count): i + 1, 0:].values)) #List of samples as arrays, ordered
        PDB_codes_count = 0 #reset count
      

print(f"{Group_df['PDB code'].nunique() -1} unique proteins")
# df.to_csv('order.csv', index = False)
        

841 unique proteins


In [41]:
print(f"{Group_df}\n{Group_df}")

     PDB code  Binding  SProtFT0  SProtFT1  SProtFT2  SProtFT3  SProtFT4  \
2400     13gs      0.0       0.0       0.0       0.0       0.0       0.0   
671      13gs      0.0       0.0       0.0       0.0       0.0       0.0   
1009     1a02      0.0       0.0       0.0       0.0       0.0       0.0   
2334     1a22      0.0       0.0       0.0       0.0       0.0       0.0   
3240     1a22      0.0       0.0       0.0       0.0       0.0       0.0   
...       ...      ...       ...       ...       ...       ...       ...   
1707     6pax      0.0       0.0       0.0       0.0       0.0       0.0   
2970     6pax      0.0       0.0       0.0       0.0       0.0       0.0   
2126     6pax      0.0       0.0       0.0       0.0       0.0       0.0   
2972     6pax      0.0       0.0       0.0       0.0       0.0       0.0   
1975     6rlx      1.0       0.0       0.0       0.0       0.0       0.0   

      SProtFT5  SProtFT6  SProtFT7  ...  NLargest6  NLargest7  NLargest8  \
2400       

In [38]:
PDB_codes_list[0]

array([['13gs', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, -18.417, 9.216, 0.431920922, 20.0, 0.0, 0.19, 0.0,
        0.0, 0.0, 196.165, 150.631, 115.221, 110.327, 82.948, 72.924,
        72.765, 71.665, 70.664, 64.469, 196.165, 150.631, 115.221,
        98.902, 82.948, 72.924, 72.765, 71.665, 70.664, 64.469, -4.19,
        -100.0, -100.0, 0.0, 0],
       ['13gs', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 80.419, 0.769317406, 20.0, -0.29, 0.0, 0.0,
        0.0, 38.925, 157.24, 146.405, 113.634, 112.582, 84.237, 81.872,
        71.952, 63.934, 61.222, 59.101, 196.165, 150.631, 115.221,
        98.902, 82.948, 72.924, 72.765, 71.665, 70.664, 64.469, -0.29,
        -100.0, -100.0, 0.0, 0]], dtype=object)

In [378]:
Input_CV = df.drop('dataset_pd', axis =1)
Output_CV = df['dataset_pd'].copy().astype('int32')
groups = df['PDB code']

# Input_train, Input_test, Classes_train, Classes_test = train_test_split(Input_CV, Output_CV, train_size = 0.8, stratify=Output_CV)

In [381]:
group_kfold = GroupKFold(n_splits=5)
group_kfold.get_n_splits(Input_train, Classes_train, groups)
print(group_kfold)

GroupKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(group_kfold.split(Input_train, Classes_train, groups)):
    
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}, group={groups[train_index]}")
    print(f"  Test:  index={test_index}, group={groups[test_index]}")

GroupKFold(n_splits=5)


ValueError: Found input variables with inconsistent numbers of samples: [2153, 2153, 2692]

### Balancing

In [ ]:
def find_minority_class(classData):
    """ Input:    classData  Array of class labels
        Returns:  minClass   The label for the minority class
                  minSize    The number of items in the minority class
                  maxSize    The number of items in the majority class

        Finds information about the inbalance in class sizes
    """
    
    Minority_count = 0
    Majority_count = 0
    for datum in classData:
        if datum == 1:
            Majority_count += 1
        elif datum == 0:
            Minority_count += 1

    minClass = 0
    minSize = Minority_count
    maxSize = Majority_count
    if Minority_count > Majority_count:
        minClass = 1
        minSize = Majority_count
        maxSize = Minority_count

    return minClass, minSize, maxSize

In [ ]:
def Balance_ratio(maxSize, minSize): 
    """ Input:      maxSize     The number of items in the majority class
                    minSize     The number of items in the minority class

        Returns:    BF          Number of balancing folds

        Calculate the number of balancing folds needed using ratio of majority to minority class size. Double to ensure sufficient
        majority class instances are sampled, then + 1 to make odd to allow weighted vote.
    """
    Divide = maxSize/minSize
    BF = (2 * round(Divide)) + 1 #Double ratio to nearest integer
    return BF

In [ ]:
def balance(inData, classData, minClass, minSize):
    """ Input:    inData          array of input data
                  classData       array of classes assigned
                  minorityClass   class label for the minority class
                  minoritySize    size of the minority class

        Returns: array of indexes that are of interest for a balanced dataset

        Perform the actual balancing between SNPs and PDs
    """
    usedLines = [False] * len(inData) #Array of false for length of data
    for i in range(len(inData)):
        if classData[i] == minClass:
            usedLines[i] = True
            
    usedCount = 0
    while usedCount < minSize:
        i = rd.randrange(len(inData))
        if usedLines[i] == False:
            usedCount += 1
            usedLines[i] = True       

    return usedLines

In [ ]:
def balance_data(inData, classData, usedLines):
    """     Input:     inData      array of input training data
                       classData   array of classes assigned to training data
                       usedLines   array of line indexes to print

            Returns:   Input_balance  Array of balanced input training data
                       Label_balance  Array of balanced classes assigned to training data
        Create arrays for the input training data and classes, as needed for predicting the probability.
        The index [i] is the identifier between the two arrays
    """
    Input_balance = []
    Label_balance = []
    for i in range(len(inData)):
        if usedLines[i]:
            Input_balance.append(inData[i])
            Label_balance.append(classData[i])
            
    Input_balance = np.stack(Input_balance, axis =0)
    Label_balance = np.stack(Label_balance, axis =0)
    
    return Input_balance, Label_balance

### Balance for n folds

In [ ]:
def Balance_Folds(BF, usedLines, Input_balance, Label_balance):
    """ Input:      BF                Number of balancing folds needed
                    usedLines         Array of line indexes to print
                    Input_balance     Input_balance  Array of balanced input training data
                    Label_balance     Array of balanced classes assigned to training data

        Returns:    Input_folds       List of 5 balanced arrays of training data
                    Output_folds      List of 5 balanced arrays of training data's labels

        Perform the balance_data() function n number of balancing fold times. Return lists for training data and labels
        where each item is the output of balance_data()
    """
    Input_folds = []
    Output_folds = []
    for fold in range(BF):
        Input_folds.append(Input_balance)
        Output_folds.append(Label_balance)


    with open('Balanced training data.txt', 'w') as f:
        for number, fold in zip(range(BF), Input_folds):
            f.write(f"Fold: {number}\n\n{fold}\n\n\n")
        
    return Input_folds, Output_folds

### RFC hyperparameter tuning

In [ ]:
 def Hyperparameter(BF, Input_folds, Output_folds):
    """ Input:      BF                Number of balancing folds needed
                    Input_folds       List of 5 balanced arrays of training data
                    Output_folds      List of 5 balanced arrays of training data's labels

        Returns:    BF_RFC_HP         List of optimized hyperparameters for each RFC

        Perform RandomSearchCV on each RFC to optimize number of trees, max depth and max samples
    """  
    estimator = RandomForestClassifier()
    param_grid = {
                'n_estimators':np.arange(50,500,50),
                'max_depth': np.arange(2, 10, 2),
                'max_samples': np.arange(0.2, 1.2, 0.2)
                  }
    BF_RFC_HP = []

    for i in range(BF):
        HPtuning = RandomizedSearchCV(
            estimator,
            param_grid, 
            scoring = 'balanced_accuracy',
            cv = 10,
            n_jobs = 6, #how many cores to run in parallel
            verbose = 2
            ).fit(Input_folds[i], Output_folds[i].ravel())
        BF_RFC_HP.append(HPtuning.best_params_)
    
    return(BF_RFC_HP)

### Train RFC on balanced dataset

In [ ]:
def BF_training(BF, Input_folds, Output_folds, BF_RFC_HP): 
    """ Input:      BF              Number of balancing folds
                    Input_folds     List of 5 balanced arrays for training data
                    Output_folds    List of 5 balanced arrays of training data's labels

        Returns:    BF_RFC          List of RFC's trained on data in each balancing fold

        Create a model that returns probability predictions for each fold, using Balance_Fold() as input
    """    
    BF_RFC = []
    
    for i in range(BF):
        BF_RFC.append(RandomForestClassifier(
                                             verbose = 1
                                            )) #Generates a RF for each fold 
        BF_RFC[i].fit(Input_folds[i], Output_folds[i].ravel()) #Fits the RFC to balanced training data    
        
    return BF_RFC

#### Test RFC on test set

In [ ]:
def BFC_test(BF_RFC, Input_test):
    """ Input:  BF_RFC          List of RFC's trained on data in each balancing fold
                Input_test      20% unseen testing data split before the balancing folds
                
        Returns:Prob_matrix     List of arrays. Each item is 2D matrix where the 1st dimension is each subset in balancing fold, 
                                2nd dimension is predicted probability
    
        Test the trained RFCs on the test set, then for every instance, outputs the predicted probability for each class
    """
    Prob_matrix = [] #Empty list
    Prob_matrixlist = []
    for i in range(len(BF_RFC)):
        Prob_list = BF_RFC[i].predict_proba(Input_test.values)
        Prob_matrix.append(Prob_list)   
        
    with open('Test probabilities.txt', 'w') as f:
        for number, line in zip(range(BF), Prob_matrix ):
            f.write(f"Fold: {number}\n\n   SNP    PD\n{line}\n\n\n")

    return Prob_matrix

### Weighted voting

In [ ]:
def Weighted_Vote(Prob_matrix, BF):
    """ Input:      Prob_matrix     List of arrays. 2D matrix where the 1st dimension is each subset in balancing fold, 
                                    2nd dimension is predicted probability
                    BF              Number of balancing folds

        Returns:    Final_vote      Weighted vote classification

        Calculate the final weighted vote using confidence scores (Sc). Binary classification formula Sc = 2|S0 - 0.5|
    """
    Sc_PD = [] #Empty list
    Sc_SNP = [] #Empty list
    for i in range(BF):
        Sc_PD.append(2* (Prob_matrix[i][:,1] - 0.5)) #Confidence scores for PD, for each fold
        Sc_SNP.append(2*(Prob_matrix[i][:,0] - 0.5)) #Confidence scores for SNP, for each fold
    

    Sum_PD = np.sum(Sc_PD, axis = 0) #Sum of all PD confidence scores. 1D Array
    Sum_SNP = np.sum(Sc_SNP, axis = 0) #Sum of all SNP confidence scores. 1D Array     

    Vote_arr = [] #Empty list
    
    for i in range(len(Classes_test)):
        if Sum_PD[i] >= Sum_SNP[i]:
            Vote_arr.append([1]) #Append PD classifications to list
        elif Sum_SNP[i] > Sum_PD[i]:
            Vote_arr.append([0]) #Append SNP classifications to list
            
    Final_vote = np.stack(Vote_arr) #Converts list of arrays to a 2D array, shape (674,1)
    Final_vote = Final_vote.ravel() #Flattens 2D array to 1D array
        
    return(Final_vote, Sum_PD, Sum_SNP) #Returns the final confidence scores


### Final confidence

In [ ]:
def Final_score(Sum_PD, Sum_SNP, BF):
    """ Input:      Sum_PD      Sum of confidence score for PD predictions
                    Sum_SNP     Sum of confidence score for SNP predictions

        Returns:    S_out        Final confidence score

        Calculate the final confidence score
    """
    
    S_Out = np.abs((Sum_PD - Sum_SNP) /(BF*2))
    np.savetxt('S_out.txt', S_Out, "%.3f")
    
    return S_Out


In [ ]:
def evalutation(Classes_test, Final_vote, S_Out):
    """ Input:      Classes_test       Class label test data
                    Final_vote         Weighted vote classification

        Evaluation metrics from RFC on test data with
    """
    Output_pred = Final_vote
    print(f"              ***Final Evaluation***\n")
    print(f"Confusion Matrix:\n {confusion_matrix(Classes_test, Output_pred)}")
    print(f"{classification_report(Classes_test, Output_pred)}\nMCC                {matthews_corrcoef(Classes_test, Output_pred)}")
    
    print(f"See file 'Classification.txt' for final classifications and confidence scores")
    np.savetxt('Classification.txt',
           np.column_stack([Final_vote, S_Out]),
           fmt = ["%.0f","%.3f"],
           delimiter ="      ",
           header = "Final classifications and confidence scores\n\n"
          )
    

### Main Program

In [ ]:

file                         = 'E2.csv'
Input, Output                = Clean_data(file)
RFC, Input_test, Classes_test, Input_train, Classes_train = train(Input, Output)
test(RFC,Input_test, Classes_test)

inData                       = pd.DataFrame(Input_train).to_numpy()
classData                    = pd.DataFrame(Classes_train).to_numpy()

minClass, minSize, maxSize   = find_minority_class(classData)
BF                           = Balance_ratio(maxSize, minSize)
usedLines                    = balance(inData, classData, minClass, minSize)

Input_balance, Label_balance = balance_data(inData, classData, usedLines)
Input_folds, Output_folds    = Balance_Folds(BF, usedLines, Input_balance, Label_balance)

BF_RFC_HP                    = Hyperparameter(BF, Input_folds, Output_folds)
BF_RFC                       = BF_training(BF, Input_folds, Output_folds, BF_RFC_HP)
Prob_matrix                  = BFC_test(BF_RFC, Input_test)

Final_vote, Sum_PD, Sum_SNP  = Weighted_Vote(Prob_matrix, BF)
S_Out                        = Final_score(Sum_PD, Sum_SNP, BF)

evalutation(Classes_test, Final_vote,S_Out)
